In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys

sys.path.append("/nas/ckgfs/users/minhpham/workspace/seed")
sys.path.append("/nas/ckgfs/users/minhpham/workspace/BLINK")

In [10]:
from seed.pipeline.retriever import Retriever

In [25]:
retriever = Retriever.get("dpr")

Attempting to initialize pre-built index wikipedia-dpr.
/nas/home/minhpham/.cache/pyserini/indexes/index-wikipedia-dpr-20210120-d1b9e6.c28f3a56b2dfcef25bf3bf755c264d04 already exists, skipping download.
Initializing wikipedia-dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Attempting to initialize pre-built index wikipedia-dpr-multi-bf.
/nas/home/minhpham/.cache/pyserini/indexes/dindex-wikipedia-dpr_multi-bf-20200127-f403c3.29eb39fe0b00a03c36c0eeae4c24f775 already exists, skipping download.
Initializing wikipedia-dpr-multi-bf...
Attempting to initialize pre-built index wikipedia-dpr.
/nas/home/minhpham/.cache/pyserini/indexes/index-wikipedia-dpr-20210120-d1b9e6.c28f3a56b2dfcef25bf3bf755c264d04 already exists, skipping download.
Initializing wikipedia-dpr...


In [11]:
retriever2 = Retriever.get("wiki")

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-27 04:04:51,231 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-27 04:04:55,224 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [31]:
import pandas as pd
import jsonlines
import editdistance

mrr = 0
recall_at_10 = 0

df = pd.DataFrame(list(jsonlines.open("../data/tabpert/processed/test.jsonl")))
for i, obj in df.iterrows():
    table = pd.DataFrame([obj["table"]])
    table = table.applymap(lambda x: " ; ".join(x) if isinstance(x, list) else x)
    label = obj["label"]
    documents = retriever2.search(table)
    for idx, document in enumerate(documents):
        if document["title"] == obj["title"]:
            recall_at_10 += 1
            mrr += 1 / (idx + 1)
            break
    if i % 1000 == 0:
        print(f"MRR: {mrr / (i + 1)}")
        print(f"Recall@10: {recall_at_10 / (i + 1)}")

MRR: 0.0
Recall@10: 0.0
MRR: 0.4366258270167486
Recall@10: 0.8541458541458542
MRR: 0.4353812110862796
Recall@10: 0.8470764617691154
MRR: 0.43465110365906545
Recall@10: 0.8533822059313562
MRR: 0.4299644221033001
Recall@10: 0.8515371157210697
